In [111]:
import Pkg

Pkg.add("JLD2")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed FileIO ─ v1.16.3
   Installed JLD2 ─── v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [033835bb] + JLD2 v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [5789e2e9] + FileIO v1.16.3
  [033835bb] + JLD2 v0.4.51
Precompiling project...
  ✓ FileIO
  ✓ Plots → FileIOExt
  ✓ JLD2
  ✓ TaylorSeries → TaylorSeriesJLD2Ext
  4 dependencies successfully precompiled in 79 seconds. 538 already precompiled.


In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR
using JLD2

In [23]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1983,01,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

492×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1983-01-01 to 2023-12-01
┌────────────┬────────────────────┬──────────────┬─────────────────┬────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI   │ Yeld_10Y_T_Bill │ CPI       ⋯
├────────────┼────────────────────┼──────────────┼─────────────────┼────────────
│ 1983-01-01 │           0.014489 │   0.00528335 │         0.00245 │   0.00102 ⋯
│ 1983-02-01 │          0.0310726 │   0.00649825 │          0.0018 │           ⋯
│ 1983-03-01 │          0.0340539 │ -0.000441667 │      0.00189167 │   0.00712 ⋯
│ 1983-04-01 │          0.0363394 │  0.000508547 │      0.00171667 │   0.00606 ⋯
│ 1983-05-01 │          0.0105384 │   0.00354027 │      0.00218333 │   0.00301 ⋯
│ 1983-06-01 │       -0.000244417 │   0.00300873 │      0.00180833 │   0.00401 ⋯
│ 1983-07-01 │         -0.0317932 │   0.00424411 │      0.00223333 │    0.0029 ⋯
│ 1983-08-01 │           0.024758 │    0.0024634 │          0.0022 │   0.00497 ⋯
│     ⋮      │         ⋮         

In [24]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,180, 100_000)
returns_scenarios = scenarios[1:2,:,:]

2×100000×181 Array{Float64, 3}:
[:, :, 1] =
  0.0243153    0.0243153    0.0243153   …   0.0243153    0.0243153
 -0.00117661  -0.00117661  -0.00117661     -0.00117661  -0.00117661

[:, :, 2] =
 -0.0252754    0.0179326   -0.00836976  …  0.00502068   0.0844053
  0.00542906  -0.00206516   0.00437032     0.00368939  -0.000667171

[:, :, 3] =
 -0.0230817   0.0252441   0.0317432   0.0509063   …  -0.00487671  0.059377
  0.00265122  0.00423268  0.00123074  0.00146327     -4.02853e-6  0.00237524

;;; … 

[:, :, 179] =
  0.0951308   -0.0132981    0.00295992  …  0.0077212    0.0180104
 -0.00436496  -0.00304255  -0.00246368     0.00199969  -0.00258319

[:, :, 180] =
  0.0576857   -0.0188951   -0.00150595  …  0.0231306    0.0260888
 -0.00222107  -0.00326863   0.00300937     0.00266792  -0.00578045

[:, :, 181] =
  0.0464981   -0.0347365   -0.0331284    …  -0.0520567    0.00587292
 -0.00121029  -0.00163864  -0.000404102     -0.00247795  -0.00594805

In [16]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [25]:
returns_scenarios_ann = zeros(2,100_000,15)
for asset in 1:2
    returns_scenarios_ann[asset,:,:] = exp.(annualise(returns_scenarios[asset,:,2:end],12))
end
returns_scenarios_ann


2×100000×15 Array{Float64, 3}:
[:, :, 1] =
 1.17452  1.12663  1.28644  1.24393   …  1.5883  0.856571  0.985756  1.3853
 1.03179  1.00588  1.01336  0.997625     1.0396  1.02651   1.0094    1.01994

[:, :, 2] =
 1.17304   1.10423  1.02855  1.04494  …  0.857618  1.44583  1.15859  1.11037
 0.999958  1.00915  1.01335  1.00618     1.00822   1.02057  1.0044   1.03129

[:, :, 3] =
 1.24631  1.06766  1.17794  0.943038  1.14259  …  0.799326  1.1322   0.902975
 0.9803   1.02072  1.01921  1.02238   1.00975     1.0104    1.02719  1.01143

;;; … 

[:, :, 13] =
 0.983737  1.32544   0.902133  1.34704   …  1.07126   0.942498  1.1444
 1.00121   0.993948  0.972342  0.987607     0.991005  0.984022  0.974334

[:, :, 14] =
 1.04993   1.14308  1.26607   1.29261   …  0.969888  1.15372   1.13011
 0.966338  1.00395  0.980975  0.976417     1.00295   0.950929  0.98691

[:, :, 15] =
 1.37546   1.00687   1.16326   1.19325  …  0.9243   0.922061  0.867845
 0.974418  0.985744  0.968787  1.00494     1.01214  0.968912  

In [26]:
res = generate_lattice(returns_scenarios_ann, 11,15)


Lattice([1.3296716695744188 1.5128215797475306 … 1.2590512006232488 0.9297488368720059; 1.1093869602541837 1.24811208166725 … 1.3085768366492514 1.047328203105469; … ; 1.1712583487973307 1.3848122087800605 … 1.4509568813860538 1.1744457361737497; 1.1370050506720266 1.3850321196840132 … 1.4506064580158853 1.060956215903349;;; 0.9915541231182996 1.0110310017716893 … 1.011756646498602 1.0132120012381172; 1.0290075807872758 1.0127210357141785 … 0.9881226388526166 1.0140760249898435; … ; 0.9647614583960772 1.0145636451161935 … 0.9794109851561821 1.0103221833146145; 1.0359752399713316 1.0141553500587719 … 0.9789159269485171 0.9899699572156789], [[1.0;;], [0.06855 0.04212 … 0.11928 0.11705], [0.10940919037199125 0.12049598832968636 … 0.06564551422319474 0.14252370532458059; 0.11562203228869895 0.12725546058879392 … 0.06623931623931624 0.13580246913580246; … ; 0.11384976525821597 0.1255868544600939 … 0.06773977196512408 0.1403420523138833; 0.11268688594617685 0.12575822298163178 … 0.0706535668

In [27]:
round.(res.states[3,:,:]; digits=4)

11×2 Matrix{Float64}:
 1.2515  1.0004
 0.8816  1.0097
 0.952   0.9863
 1.3979  1.0222
 1.0699  1.0021
 1.1495  1.0409
 1.1883  1.0183
 1.025   1.0213
 0.9086  1.0375
 1.4608  0.9925
 1.1747  0.9798

In [28]:
round.(res.states[4,:,:]; digits=4)

11×2 Matrix{Float64}:
 0.8903  1.0188
 1.0577  0.9777
 1.2409  1.0367
 1.0263  1.0032
 1.4907  1.008
 1.3291  0.9821
 0.9777  1.0413
 0.8852  0.994
 1.1705  0.9968
 1.2678  1.0122
 1.0879  1.0201

In [29]:
round.(res.probs[4][1,:]; digits=2)

11-element Vector{Float64}:
 0.11
 0.09
 0.07
 0.06
 0.15
 0.08
 0.13
 0.13
 0.08
 0.04
 0.06

In [30]:
save_object("scenario_grid.jld2", res)

In [60]:
grid = load_object("scenario_grid.jld2")

Lattice([1.0853933315390984 1.218288162475466 … 1.8355566846831484 1.4009515471553637; 0.94539648550184 0.9352273412884807 … 1.5265792998503565 1.2383234954409708; … ; 0.9191554431977321 1.154816879403226 … 1.3300414893432977 1.038763668706857; 0.9261635028461954 1.7917767202361508 … 1.1708527976718917 1.3322260375257169;;; 0.9765946310583116 1.1774415120338133 … 0.9291402188272971 0.8899189392863953; 1.2181816749731287 1.0059295736387661 … 1.0865583310648845 0.8873290097745962; … ; 1.1156717683855484 0.9243580513031538 … 1.100655126873907 1.0823987352357525; 1.109849303328651 0.9550545807263788 … 0.9243075236913665 1.0942922799016646;;; 1.0402671160866452 1.0463306482683974 … 1.0228155547306848 1.0562637488834923; 1.0101162372306656 0.9790255920467821 … 1.0822322232049224 0.9539788581889932; … ; 0.9764644843180704 1.0088132672878474 … 0.9857306066451239 1.0572844235140704; 0.9713402918750139 1.0025639047032668 … 1.0070728088493606 0.9795746837072669], [[1.0;;], [0.157 0.10318 … 0.0706

In [38]:
using  Test
@test grid.states[5,:,:] ≈ res.states[5,:,:]

Test Passed

In [41]:
res.states[3,:,3]

9-element Vector{Float64}:
 1.0143305473204354
 0.9854785572603496
 1.003604484079091
 1.0038924296581662
 1.0107697123646282
 0.9910509301591774
 1.0332650815623272
 1.0259226703370474
 1.034748568861909

scenarios_moments

t | mean | std | var | skewness| kurtosis | 
--- | --- | --- | --- | --- | --- |

print moments

lattice_moments

#### TODO

- evaluate moments